## <p style="text-align: center;">BA108_2122_TMD2_CA_2</p>
### <p style="text-align: center;"> Details </p>
#### <p style="text-align: center;">Name: Sunil Judhistira Gauda</p>
#### <p style="text-align: center;">Student ID : 10595858 </p>
##### <p style="text-align: center;">Github : https://github.com/sunil3092/BA108_2122_TMD2_CA2 </p>

In [126]:
import requests as req
from bs4 import BeautifulSoup
import re
import time
import sqlite3

In [127]:
def manage_tags(html, tags):
    soup = BeautifulSoup(html, "html.parser")

    for page_data in soup(tags):
        page_data.decompose()

    return ''.join(soup.stripped_strings)

In [128]:
top_rated_list = []

In [129]:
page_number = [1,2,3,4,5]


In [130]:
for num in page_number:
     url = "https://www.swiggy.com/city/mumbai/top-rated-collection?page=" + str(num)
     request = req.get(url)
     soup = BeautifulSoup(request.content)
     div = soup.find_all("div", {"class" : "_3FR5S"})
     time.sleep(10)
     for i in range( len(div)):
          top_rated = {}
          top_rated["name"] = div[i].find("div", {"class" : "nA6kb"}).text
          top_rated["type"] = div[i].find("div", {"class" : "_1gURR"}).text
          top_rated["rating"] = float(re.sub('\D','',div[i].find("div", {"class" : "_9uwBC"}).text))
          stringValue = div[i].find("div", {"class" : "_3Mn31"}).text
          split_string = stringValue.split("•")
          top_rated["deliviery_time"] = int(re.sub('\D','',split_string[1]))
          top_rated["price_for_two"] = int(re.sub('\D','',split_string[2]))
          discount = div[i].find("span", {"class" : "sNAfh"})
          if discount is not None:
               discount = discount.text
               if len(discount) > 0:
                    top_rated["discount"] = int(re.sub('\D','', discount.split("|")[0]))
                    top_rated["discount_code"] = re.sub(" ","",discount.split("|")[1].replace("Use ", ""))
               else:
                    top_rated["discount"] = 0
                    top_rated["discount_code"] = ""
          top_rated_list.append(top_rated)

In [131]:
use_sqlite3 = True


In [149]:
def create_db(use_sqlite3):
    conn = sqlite3.connect('swiggytop.db')
    if use_sqlite3:       
            conn.execute('''
                CREATE TABLE IF NOT EXISTS SWIGGYTOP
                    (
                        NAME TEXT NOT NULL,
                        TYPE TEXT NOT NULL,
                        RATING REAL NOT NULL,
                        DELIVIERY_TIME INT NOT NULL,
                        PRICE_FOR_TWO REAL NOT NULL,
                        DISCOUNT REAL,
                        DISCOUNT_CODE TEXT
                    );
            ''')
            print("Table Created")
            conn.close()

In [152]:
def write_data(data):
    conn = sqlite3.connect('swiggytop.db')
    for value in  data:
        query = f'''
                INSERT INTO SWIGGYTOP
                (
                    NAME,
                    TYPE,
                    RATING,
                    DELIVIERY_TIME,
                    PRICE_FOR_TWO,
                    DISCOUNT,
                    DISCOUNT_CODE)
                VALUES({value["name"]},{value["type"]},{value["rating"]},{value["deliviery_time"]},{value["price_for_two"]},{value["discount"]},{value["discount_code"]})    
                '''
        cur = conn.cursor()
        cur.execute(query)
        conn.commit()
    conn.close()
    return cur.lastrowid

In [153]:
create_db(True)
write_data(top_rated_list)

Table Created


OperationalError: near "Waffle": syntax error

In [ ]:
conn.execute("SELECT * FROM SWIGGYTOP")
cursor = conn.cursor()
cursor.fetchall()